# 1.  Installations and Settings 🛠️

In [6]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu

# 2.  Setting up your LLM 🧠

In [7]:
from langchain_huggingface import HuggingFaceEndpoint

# This info's at the top of each HuggingFace model page
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(repo_id = hf_model)

# 3.  Retrieval Augmented Generation 🔃

## 3.1 loading Data

In [ ]:
AsyncChromiumLoader

In [8]:
# load dependancies
from langchain_community.document_loaders import AsyncHtmlLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
urls = [
    "https://www.daad.de/en/studying-in-germany/scholarships/daad-scholarships/",
    "https://www.daad.de/en/studying-in-germany/universities/the-right-degree-programme/", 
    "https://www2.daad.de/deutschland/stipendium/datenbank/en/21148-scholarship-database/", 
    "https://www2.daad.de/deutschland/stipendium/datenbank/en/21148-scholarship-database/?status=&origin=&subjectGrps=&daad=&intention=&q=&page=1&detail=57742130",
    "https://www2.daad.de/deutschland/stipendium/datenbank/en/21148-scholarship-database/?status=&origin=&subjectGrps=&daad=&intention=&q=&page=1&detail=57135739",
    "https://www2.daad.de/deutschland/stipendium/datenbank/en/21148-scholarship-database/?status=&origin=&subjectGrps=&daad=&intention=&q=&page=1&detail=57507783",
    "https://www.daad.de/en/studying-in-germany/universities/universities/",
    "https://www.daad.de/en/studying-in-germany/universities/haw/",
    "https://www.daad.de/en/studying-in-germany/universities/dual-studies/",
    "https://www.daad.de/en/studying-in-germany/scholarships/daad-scholarships/",
    "https://www.daad.de/en/studying-in-germany/scholarships/information-for-scholarship-applicants/#requirements",
    "https://www.daad.de/en/studying-in-germany/scholarships/funding-options/",
    "https://www.daad.de/en/studying-in-germany/requirements/", 
    "https://www.daad.de/en/studying-in-germany/requirements/overview/", 
    "https://www.daad.de/en/studying-in-germany/requirements/application-process/",
    "https://www.daad.de/en/studying-in-germany/requirements/enrolling/",
    "https://www.daad.de/en/studying-in-germany/requirements/studienkollegs/",
    "https://www.daad.de/en/studying-in-germany/living-in-germany/visa/",
    "https://www.daad.de/en/studying-in-germany/living-in-germany/registering/",
    "https://www.daad.de/en/studying-in-germany/living-in-germany/health-insurance/",
    "https://www.daad.de/en/studying-in-germany/living-in-germany/renting/",
    "https://www.daad.de/en/studying-in-germany/living-in-germany/german-language/learning/",
    "https://www.daad.de/en/studying-in-germany/living-in-germany/german-language/",
    "https://www.daad.de/en/studying-in-germany/living-in-germany/finances/",
    "https://www.daad.de/en/studying-in-germany/living-in-germany/safety/",
    "https://www.daad.de/en/studying-in-germany/work-career/",
    "https://www.daad.de/en/studying-in-germany/work-career/career-planning/",
    "https://www.daad.de/en/studying-in-germany/work-career/side-jobs/",
    "https://www.daad.de/en/studying-in-germany/work-career/work-placements/",
    "https://www.daad.de/en/studying-in-germany/advisory-service/publications/",
    "https://www.daad.de/en/studying-in-germany/advisory-service/psychological-wellbeing/",
    "https://www.daad.de/en/the-daad/mobility-with-a-disability/"
]

In [10]:
from langchain_community.document_loaders import AsyncHtmlLoader

loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 32/32 [00:10<00:00,  3.03it/s]


## 3.2 Splitting the document

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                               chunk_overlap=150)

docs = text_splitter.split_documents(docs)

## 3.3 Creating vectors with embeddings

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "docs"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

/Users/ottododoo/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## 3.4 Creating a vector database

In [13]:
from langchain.vectorstores import FAISS

vector_db = FAISS.from_documents(docs, embeddings)

In [14]:
vector_db.save_local("docs")

In [15]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3}), # top 2 results only, speed things up

In [16]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key = 'chat_history',
                                  return_messages = True,
                                  output_key = 'answer')

In [17]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=hf_model)

embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "docs"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

vector_db = FAISS.load_local("docs", embeddings, allow_dangerous_deserialization=True)

retriever = vector_db.as_retriever(search_kwargs={"k": 3})

memory = ConversationBufferMemory(memory_key = 'chat_history',
                                  return_messages = True,
                                  output_key = 'answer')  # Set output_key to 'answer'

template = """You are a nice chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short and succinct.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {question}
Response:"""

prompt = PromptTemplate(template = template,
                        input_variables = ["context", "question"])

# chain
chain = ConversationalRetrievalChain.from_llm(llm,
                                              retriever = retriever,
                                              memory = memory,
                                              return_source_documents = True,
                                              combine_docs_chain_kwargs = {"prompt": prompt})

/Users/ottododoo/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if hasattr(m, '__version__')))

streamlit==1.35.0
langchain_community==0.2.6


In [19]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Print versions of the imported modules
import streamlit as st

print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if hasattr(m, '__version__')))


streamlit==1.35.0
langchain_community==0.2.6
